In [1]:

import pandas as pd
import numpy as np
import joblib

# Load saved model and preprocessing objects
best_model = joblib.load('best_speed_model.pkl')
scaler = joblib.load('scaler.pkl')
label_encoders = joblib.load('label_encoders.pkl')
target_cols = joblib.load('target_cols.pkl')
non_numeric_cols = joblib.load('non_numeric_cols.pkl')

# Load new data (with UNICO inside)
file = r"C:\Users\Salma\Desktop\Data_ne\data_new.xlsx"
df = pd.read_excel(file)

# Drop irrelevant columns (optional - must match training preprocessing)
irrelevant_cols = [
    'FAM', 'FIL', 'KIT_G', 'KIT_D', 'TER_D', 'OUTIL_D',
    'SEAL_D', 'OUTIL_G', 'SealKey', 'TerminalKey', 'WireKey', 'Projet'
]
df = df.drop(columns=[col for col in irrelevant_cols if col in df.columns])

# Prediction function
def predict_speed_parameters_by_unico(unico_name, df, best_model, scaler, label_encoders, target_cols, non_numeric_cols):
    row = df[df["UNICO"] == unico_name]
    if row.empty:
        print(f"UNICO '{unico_name}' not found.")
        return None

    # Drop target columns if present
    row_X = row.drop(columns=[col for col in target_cols if col in row.columns])

    # Apply label encoding
    for col in non_numeric_cols:
        if col in row_X.columns:
            le = label_encoders.get(col)
            if le:
                row_X[col] = row_X[col].astype(str).apply(lambda x: x if x in le.classes_ else 'Unknown')
                if 'Unknown' not in le.classes_:
                    le.classes_ = np.append(le.classes_, 'Unknown')
                row_X[col] = le.transform(row_X[col])

    # Align column order with training data
    row_X = row_X.reindex(columns=[col for col in scaler.feature_names_in_])

    # Apply scaling
    row_X_scaled = scaler.transform(row_X)

    # Make prediction
    prediction = best_model.predict(row_X_scaled)

    # Output
    predicted_df = pd.DataFrame(prediction, columns=target_cols)
    print(f"\nPredicted Speed Parameters for UNICO '{unico_name}':")
    print(predicted_df.T)
    return predicted_df

# Example usage
predict_speed_parameters_by_unico(
    "230_nez", 
    df, 
    best_model, 
    scaler, 
    label_encoders, 
    target_cols, 
    non_numeric_cols
)



Predicted Speed Parameters for UNICO '230_nez':
                         0
Speed                 93.6
Accel                 93.6
Deccel                93.6
SwivelSpeed_S1       100.0
SwivelSpeed_S2       100.0
SwivelSpeed_S1_seal   80.0
SwivelSpeed_S2_seal   80.0
SwivelSpeed_S1term   100.0
SwivelSpeed_S2term   100.0
ProcessingSpeed_S1   100.0
ProcessingSpeed_S2   100.0
PressSpeed           100.0


,Speed,Accel,Deccel,SwivelSpeed_S1,SwivelSpeed_S2,SwivelSpeed_S1_seal,SwivelSpeed_S2_seal,SwivelSpeed_S1term,SwivelSpeed_S2term,ProcessingSpeed_S1,ProcessingSpeed_S2,PressSpeed
0,93.6,93.6,93.6,100.0,100.0,80.0,80.0,100.0,100.0,100.0,100.0,100.0
